In [ ]:
import pandas as pd
import time
from utils.fillData import getFirstToken, searchByToken, get_uuid_from_nan_values

In [ ]:
lang_labels = ['Spanish','Portuguese (Brazil)', 'Korean', 'Japanese', 'French', 'German', 
         'Chinese Traditional', 'Chinese Simplified']

In [ ]:
cards = pd.read_csv("csv/cards.csv")
translate = pd.read_csv("csv/foreign_data.csv")

In [ ]:
uuid = list(cards['uuid'])
uuid_first_token = [getFirstToken(x, '-') for x in uuid]
print(uuid[0])
print(uuid_first_token[0])

# Main Goal:

Fill text fields without data (nan) with csv data in english and find the match between cards according to their language and their uuid

In [ ]:
def match_Data(df_lang, attribute, list_ids, uuids): 
    start = time.time()
    for i in range(len(list_ids)):
        index_esp = list_ids[i]
        first_token = uuids[i]
        index_eng = searchByToken(uuid_first_token, first_token)[0]
        _card = cards.iloc[index_eng]
        df_lang[attribute].iloc[index_esp] = _card[attribute]
    end = time.time()
    print ('time:', end - start)

In [ ]:
def invoke_match_Data(df_lang):
    for attribute in ['type','text', 'flavorText']:
        print('MATCHING ', attribute)
        LIST_ATTR = list(df_lang[attribute])
        ids, uuids = get_uuid_from_nan_values(LIST_ATTR, df_lang)
        match_Data(df_lang, attribute, ids, uuids)

In [ ]:
def nullableTest(df, attribute):
    e = list(df[attribute])[0:100]
    for i in range(len(e)):
        if (type(e[i]) == float):
            _test = df.iloc[i]
            r = searchByToken(uuid_first_token, getFirstToken(_test['uuid'],'-'))[0]
            _card = cards.iloc[r]
            print(i, e[i], _test['uuid'],' english->' ,_card['id'], _card[attribute], _card['uuid'])

In [ ]:
def df_tester(df_lang):
    for attribute in ['type','text', 'flavorText']:
        print('Testing: ', attribute)
        print('\nid_lang, attribute, uuid_lang, || english_id, english_atribute, english_uuid')
        nullableTest(df_lang, attribute)

## Match databases: 

### ADD INDEX CARDS(ORIGINAL) TO INDEX IN LANGUAGE

### Why match indexes?

If you are looking for a card with a language other than English, you should be aware that the dataset of the chosen language does not contain all the data, so you should link to the original dataset in English.

In [ ]:
def tokenize(series):
    start = time.time()
    tokens = list(series)
    data = [getFirstToken(x, '-') for x in tokens]
    
    end = time.time()
    print('tokenize time: ',end -start)
    return data

def matchingIndexByUUID(tokenized_list):
    start = time.time()
    data = [searchByToken(uuid_first_token, x)[0] for x in tokenized_list]
    end = time.time()
    print('matchingIndexByUUID time: ', end - start)
    return data

### Export to csv files

In [ ]:
def export_csv(df_lang, df_lang_index):
    df_lang.to_csv('./output/%s_cards.csv'%(lang_label),index = False, header=True)
    df_lang_index.to_csv('./output/%s_indexs_matches.csv'%(lang_label),index = False, header=True)
    print('data exported to csv!')

## RUN!

### Filling and Matching all langs data!

In [ ]:
for lang_label in lang_labels:
    print('\nMatching in ', lang_label)
    _lang = translate[translate['language']==lang_label]
    invoke_match_Data(_lang)
    uuid_lang = _lang['uuid']
    UUID_LANG_TOKEN = tokenize(uuid_lang)
    INDEX = matchingIndexByUUID(UUID_LANG_TOKEN)
    df_INDEX = pd.DataFrame(INDEX, columns = ['match'])
    export_csv(_lang, df_INDEX)

## Search mode

Find the index of the card (or possible cards) according to the input parameter. For each index, the basic values in the language will be obtained by consulting the corresponding dataset. For the other values, you must examine the csv: *_indexs_matches.csv, and with the index search in the original dataset.